In [8]:
## Run this cell to get unjumbled video. Other cells are just how i made this code step by step
## MAIN CELL

import cv2
import os
import numpy as np
import torch
import torch.nn.functional as F
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time
from collections import defaultdict
import multiprocessing as mp

# ===============================
# Configuration
# ===============================
ANALYSIS_RESIZE = (320, 180)  # Reduced resolution
SSIM_SIZE = 7
SSIM_SIGMA = 1.5
SSIM_CHANNELS = 3
SSIM_PAD = SSIM_SIZE // 2
NUM_WORKERS = mp.cpu_count() * 2  # 2x CPU cores for I/O bound tasks

# ===============================
# Device
# ===============================
device = torch.device("cpu")
print(f"✅ Using device: {device} with {NUM_WORKERS} threads")

# ===============================
# Gaussian Kernel for SSIM
# ===============================
def gaussian_kernel(size, sigma, channels):
    coords = torch.arange(size).float() - size / 2 + 0.5
    gauss = torch.exp(-(coords ** 2) / (2 * sigma ** 2))
    gauss /= gauss.sum()
    kernel_1d = gauss.unsqueeze(1)
    kernel_2d = torch.matmul(kernel_1d, kernel_1d.t()).unsqueeze(0).unsqueeze(0)
    return kernel_2d.expand(channels, 1, size, size).contiguous()

SSIM_KERNEL = gaussian_kernel(SSIM_SIZE, SSIM_SIGMA, SSIM_CHANNELS)

def ssim_torch(img1, img2, kernel=SSIM_KERNEL):
    kernel = kernel.to(img1.device)
    C1, C2 = 0.01 ** 2, 0.03 ** 2

    def conv(img):
        return F.conv2d(img, kernel, groups=SSIM_CHANNELS, padding=SSIM_PAD)

    mu1, mu2 = conv(img1), conv(img2)
    mu1_sq, mu2_sq, mu1_mu2 = mu1 ** 2, mu2 ** 2, mu1 * mu2
    sigma1_sq = conv(img1 * img1) - mu1_sq
    sigma2_sq = conv(img2 * img2) - mu2_sq
    sigma12 = conv(img1 * img2) - mu1_mu2

    numerator = (2 * mu1_mu2 + C1) * (2 * sigma12 + C2)
    denominator = (mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2)
    return (numerator / denominator).mean().item()

# ===============================
# Utilities
# ===============================
def to_tensor(frame, device):
    return torch.from_numpy(frame).permute(2, 0, 1).unsqueeze(0).float().to(device) / 255.0

def extract_frames(video_path, output_dir="frames"):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    fps = int(cap.get(cv2.CAP_PROP_FPS) or 30)
    print(f"🎞️ Extracting frames from '{video_path}'...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imwrite(f"{output_dir}/frame_{count:06d}.jpg", frame)
        count += 1

    cap.release()
    print(f"✅ Extracted {count} frames")
    return count, fps

# ===============================
# FIXED: Fast Similarity with ThreadPoolExecutor
# ===============================
def combined_similarity_fast(args):
    """
    CPU-optimized similarity - no SSIM, fast operations
    """
    pair_idx, frames_resized = args
    i, j = pair_idx
    
    f1 = frames_resized[i]
    f2 = frames_resized[j]
    
    # 1. Fast Histogram
    h1 = cv2.calcHist([f1], [0, 1, 2], None, [6, 6, 6], [0, 256] * 3)
    h2 = cv2.calcHist([f2], [0, 1, 2], None, [6, 6, 6], [0, 256] * 3)
    cv2.normalize(h1, h1)
    cv2.normalize(h2, h2)
    hist_score = (cv2.compareHist(h1, h2, cv2.HISTCMP_CORREL) + 1) / 2
    hist_score = np.clip(hist_score, 0, 1)

    # 2. Fast Optical Flow
    gray1 = cv2.cvtColor(f1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(f2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 2, 10, 2, 5, 1.1, 0)
    flow_magnitude = np.mean(np.sqrt(flow[..., 0] ** 2 + flow[..., 1] ** 2))
    flow_score = np.exp(-flow_magnitude / 10.0)
    flow_score = np.clip(flow_score, 0, 1)

    # 3. Fast MSE
    f1_tiny = cv2.resize(f1, (80, 45))
    f2_tiny = cv2.resize(f2, (80, 45))
    mse = np.mean((f1_tiny.astype(np.float32) - f2_tiny.astype(np.float32)) ** 2)
    mse_score = np.exp(-mse / 1000.0)
    mse_score = np.clip(mse_score, 0, 1)

    # Weighted
    total = 0.35 * hist_score + 0.50 * flow_score + 0.15 * mse_score

    return i, j, total

def build_similarity_matrix_threaded(frames_resized):
    """
    Fast parallel similarity using ThreadPoolExecutor (safe for OpenCV)
    """
    n = len(frames_resized)
    sim = np.zeros((n, n))
    
    # Create pair indices
    pairs = [(i, j) for i in range(n) for j in range(i + 1, n)]
    
    print(f"🧮 Computing similarity matrix with {NUM_WORKERS} threads...")
    
    # ThreadPoolExecutor works with OpenCV
    with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
        # Pass frames_resized as part of arguments
        args_list = [((i, j), frames_resized) for i, j in pairs]
        
        results = list(
            tqdm(
                executor.map(combined_similarity_fast, args_list),
                total=len(pairs),
                ncols=100
            )
        )

    for i, j, val in results:
        sim[i][j] = sim[j][i] = val

    print("✅ Similarity matrix built")
    return sim

# ===============================
# Chain Building
# ===============================
def build_optimal_chain(similarity, lookahead=3):
    n = len(similarity)
    
    start_scores = []
    for i in range(n):
        sorted_sims = np.sort(similarity[i])[::-1]
        score = sorted_sims[0] - sorted_sims[1]
        start_scores.append(score)
    
    start_candidates = np.argsort(start_scores)[-2:]
    
    best_chain = None
    best_score = -999999
    
    print("🔍 Building optimal chain...")
    
    for start_frame in start_candidates:
        chain = [start_frame]
        visited = {start_frame}
        current = start_frame
        total_score = 0
        
        for step in range(n - 1):
            unvisited = [j for j in range(n) if j not in visited]
            if not unvisited:
                break
            
            remaining_frames = len(unvisited)
            end_game_threshold = max(20, int(0.15 * n))
            
            if remaining_frames <= end_game_threshold:
                best_next = max(unvisited, key=lambda x: similarity[current][x])
            else:
                best_next = None
                best_lookahead_score = -999999
                
                num_candidates = min(3, len(unvisited))
                candidates = sorted(unvisited, key=lambda x: similarity[current][x], reverse=True)[:num_candidates]
                
                for candidate in candidates:
                    lookahead_score = similarity[current][candidate]
                    
                    future_visited = visited | {candidate}
                    future_unvisited = [j for j in range(n) if j not in future_visited]
                    
                    if future_unvisited:
                        future_connections = [similarity[candidate][j] for j in future_unvisited]
                        top_future = sorted(future_connections, reverse=True)[:min(2, len(future_connections))]
                        lookahead_score += 0.5 * np.mean(top_future)
                    
                    if lookahead_score > best_lookahead_score:
                        best_lookahead_score = lookahead_score
                        best_next = candidate
            
            if best_next is None:
                best_next = max(unvisited, key=lambda x: similarity[current][x])
            
            chain.append(best_next)
            visited.add(best_next)
            total_score += similarity[current][best_next]
            current = best_next
        
        avg_score = total_score / (len(chain) - 1) if len(chain) > 1 else 0
        
        print(f"   Start {start_frame:03d}: Avg similarity = {avg_score:.4f}")
        
        if avg_score > best_score:
            best_score = avg_score
            best_chain = chain
    
    print(f"✅ Best chain: Start={best_chain[0]:03d}, Avg={best_score:.4f}")
    
    return best_chain

# ===============================
# Post-Processing
# ===============================
def fix_end_sequence(order, similarity, last_n=20):
    if len(order) <= last_n:
        return order
    
    print(f"\n🔧 Re-optimizing last {last_n} frames...")
    
    fixed_part = order[:-last_n]
    end_frames = set(order[-last_n:])
    
    current = fixed_part[-1]
    new_end = []
    
    for _ in range(last_n):
        if not end_frames:
            break
        best = max(end_frames, key=lambda x: similarity[current][x])
        new_end.append(best)
        end_frames.remove(best)
        current = best
    
    final_order = fixed_part + new_end
    
    old_avg = sum(similarity[order[i]][order[i+1]] for i in range(len(order) - last_n, len(order) - 1)) / (last_n - 1)
    new_avg = sum(similarity[final_order[i]][final_order[i+1]] for i in range(len(final_order) - last_n, len(final_order) - 1)) / (last_n - 1)
    
    if new_avg > old_avg:
        print(f"   ✅ Improved: {old_avg:.4f} → {new_avg:.4f}")
        return final_order
    else:
        print(f"   Already optimal: {old_avg:.4f}")
        return order

# ===============================
# Temporal Flow Validation
# ===============================
def validate_and_fix_temporal_flow(order, frames_resized, similarity, validate_last_n=30):
    if len(order) <= validate_last_n:
        validate_last_n = len(order) - 1
    
    print(f"\n🔍 Validating temporal flow (last {validate_last_n} frames)...")
    
    flow_scores = []
    bad_transitions = []
    start_idx = len(order) - validate_last_n - 1
    
    for i in range(start_idx, len(order) - 1):
        curr_frame = frames_resized[order[i]]
        next_frame = frames_resized[order[i + 1]]
        
        gray1 = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)
        
        flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 2, 10, 2, 5, 1.1, 0)
        flow_magnitude = np.mean(np.sqrt(flow[..., 0] ** 2 + flow[..., 1] ** 2))
        flow_scores.append(flow_magnitude)
        
        if len(flow_scores) >= 3:
            recent_avg = np.mean(flow_scores[-3:])
            if flow_magnitude > 2.0 * recent_avg and flow_magnitude > 15:
                bad_transitions.append((i, flow_magnitude, recent_avg))
    
    if bad_transitions:
        print(f"   ⚠️ Found {len(bad_transitions)} issues - fixing...")
        return fix_problematic_section(order, frames_resized, similarity, bad_transitions)
    else:
        print("   ✅ Flow looks good!")
        return order

def fix_problematic_section(order, frames_resized, similarity, bad_transitions):
    earliest_problem = min(pos for pos, _, _ in bad_transitions)
    fix_start = max(0, earliest_problem - 10)
    
    fixed_part = order[:fix_start]
    problem_frames = set(order[fix_start:])
    
    current = fixed_part[-1] if fixed_part else order[0]
    new_section = []
    
    for _ in range(len(problem_frames)):
        if not problem_frames:
            break
        
        best_candidate = None
        best_score = -1
        
        for candidate in problem_frames:
            sim_score = similarity[current][candidate]
            
            curr_frame = frames_resized[current]
            cand_frame = frames_resized[candidate]
            gray1 = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
            gray2 = cv2.cvtColor(cand_frame, cv2.COLOR_BGR2GRAY)
            
            flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 2, 10, 2, 5, 1.1, 0)
            flow_magnitude = np.mean(np.sqrt(flow[..., 0] ** 2 + flow[..., 1] ** 2))
            flow_penalty = np.exp(-flow_magnitude / 8.0)
            
            combined_score = 0.7 * sim_score + 0.3 * flow_penalty
            
            if combined_score > best_score:
                best_score = combined_score
                best_candidate = candidate
        
        if best_candidate is not None:
            new_section.append(best_candidate)
            problem_frames.remove(best_candidate)
            current = best_candidate
    
    return fixed_part + new_section

# ===============================
# Outlier Removal
# ===============================
def remove_outliers(order, similarity, threshold=0.10):
    n = len(order)
    clean_order = [order[0]]
    removed_count = 0
    
    for i in range(1, n):
        prev_idx = order[i - 1]
        curr_idx = order[i]
        sim_score = similarity[prev_idx][curr_idx]
        
        position_ratio = i / n
        adjusted_threshold = threshold * 0.6 if position_ratio > 0.8 else threshold
        
        if sim_score > adjusted_threshold:
            clean_order.append(curr_idx)
        else:
            removed_count += 1
    
    if removed_count > 0:
        print(f"\n⚠️ Removed {removed_count} outliers")
    else:
        print("\n✅ No outliers")
    
    return clean_order

# ===============================
# Rebuild Video
# ===============================
def rebuild_video(order, frame_dir="frames", output_file="reconstructed_video.mp4", fps=30):
    sample = cv2.imread(f"{frame_dir}/frame_000000.jpg")
    h, w, _ = sample.shape
    out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    print(f"🎬 Rebuilding video ({len(order)} frames)...")

    def load_frame(idx):
        return cv2.imread(f"{frame_dir}/frame_{idx:06d}.jpg")

    with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
        for frame in tqdm(executor.map(load_frame, order), total=len(order), desc="Writing", ncols=100):
            if frame is not None:
                out.write(frame)

    out.release()
    print(f"✅ Video saved: {output_file}")

# ===============================
# Main
# ===============================
def main(video_path):
    start_time = time.time()

    total, fps = extract_frames(video_path)

    print("📥 Loading frames...")
    frames_resized = []
    for i in tqdm(range(total), desc="Loading", ncols=100):
        frame = cv2.imread(f"frames/frame_{i:06d}.jpg")
        frame_resized = cv2.resize(frame, ANALYSIS_RESIZE)
        frames_resized.append(frame_resized)

    similarity = build_similarity_matrix_threaded(frames_resized)
    best_order = build_optimal_chain(similarity)
    best_order = fix_end_sequence(best_order, similarity, last_n=20)
    best_order = validate_and_fix_temporal_flow(best_order, frames_resized, similarity, validate_last_n=30)
    
    print(f"\n📋 First 10: {best_order[:10]}")
    print(f"📋 Last 10:  {best_order[-10:]}")
    
    if len(best_order) >= 11:
        end_sims = [similarity[best_order[i]][best_order[i + 1]] for i in range(len(best_order) - 11, len(best_order) - 1)]
        print(f"\n📊 End: Avg={np.mean(end_sims):.4f}, Min={np.min(end_sims):.4f}")
    
    clean_order = remove_outliers(best_order, similarity, threshold=0.10)
    print(f"\n📊 Final: {len(clean_order)}/{total} frames")

    rebuild_video(clean_order, fps=fps)

    print(f"\n🕒 Total: {time.time() - start_time:.2f}s")
    print("✅ Done!")

if __name__ == "__main__":
    main("jumbled_video.mp4")


✅ Using device: cpu with 32 threads
🎞️ Extracting frames from 'jumbled_video.mp4'...
✅ Extracted 300 frames
📥 Loading frames...


Loading: 100%|████████████████████████████████████████████████████| 300/300 [00:05<00:00, 58.87it/s]


🧮 Computing similarity matrix with 32 threads...


100%|████████████████████████████████████████████████████████| 44850/44850 [01:20<00:00, 556.87it/s]


✅ Similarity matrix built
🔍 Building optimal chain...
   Start 293: Avg similarity = 0.9950
   Start 096: Avg similarity = 0.9954
✅ Best chain: Start=096, Avg=0.9954

🔧 Re-optimizing last 20 frames...
   Already optimal: 0.9928

🔍 Validating temporal flow (last 30 frames)...
   ✅ Flow looks good!

📋 First 10: [np.int64(96), 177, 101, 203, 33, 249, 51, 102, 231, 141]
📋 Last 10:  [276, 38, 206, 185, 73, 0, 261, 225, 283, 118]

📊 End: Avg=0.9886, Min=0.9390

✅ No outliers

📊 Final: 300/300 frames
🎬 Rebuilding video (300 frames)...


Writing: 100%|████████████████████████████████████████████████████| 300/300 [00:07<00:00, 41.19it/s]


✅ Video saved: reconstructed_video.mp4

🕒 Total: 99.30s
✅ Done!


In [4]:
# DAY1:Setting up basic frame extraction
import cv2
import os
import numpy as np
import torch
import torch.nn.functional as F
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time
from collections import defaultdict
import multiprocessing as mp

# ===============================
# Configuration
# ===============================
ANALYSIS_RESIZE = (320, 180)
SSIM_SIZE = 7
SSIM_SIGMA = 1.5
SSIM_CHANNELS = 3
SSIM_PAD = SSIM_SIZE // 2
NUM_WORKERS = mp.cpu_count() * 2  # 2x CPU cores for I/O bound tasks

# ===============================
# Device
# ===============================
device = torch.device("cpu")
print(f"✅ Using device: {device} with {NUM_WORKERS} threads")

✅ Using device: cpu with 32 threads


In [3]:
# =====================================
# DAY 2: GPU Detection + SSIM Kernel
# =====================================
def gaussian_kernel(size, sigma, channels):
    coords = torch.arange(size).float() - size / 2 + 0.5
    gauss = torch.exp(-(coords ** 2) / (2 * sigma ** 2))
    gauss /= gauss.sum()
    kernel_1d = gauss.unsqueeze(1)
    kernel_2d = torch.matmul(kernel_1d, kernel_1d.t()).unsqueeze(0).unsqueeze(0)
    return kernel_2d.expand(channels, 1, size, size).contiguous()

SSIM_KERNEL = gaussian_kernel(SSIM_SIZE, SSIM_SIGMA, SSIM_CHANNELS)

def ssim_torch(img1, img2, kernel=SSIM_KERNEL):
    kernel = kernel.to(img1.device)
    C1, C2 = 0.01 ** 2, 0.03 ** 2

    def conv(img):
        return F.conv2d(img, kernel, groups=SSIM_CHANNELS, padding=SSIM_PAD)

    mu1, mu2 = conv(img1), conv(img2)
    mu1_sq, mu2_sq, mu1_mu2 = mu1 ** 2, mu2 ** 2, mu1 * mu2
    sigma1_sq = conv(img1 * img1) - mu1_sq
    sigma2_sq = conv(img2 * img2) - mu2_sq
    sigma12 = conv(img1 * img2) - mu1_mu2

    numerator = (2 * mu1_mu2 + C1) * (2 * sigma12 + C2)
    denominator = (mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2)
    return (numerator / denominator).mean().item()

# Convert frame to tensor
def to_tensor(frame, device):
    return torch.from_numpy(frame).permute(2, 0, 1).unsqueeze(0).float().to(device) / 255.0

# Extract frames
def extract_frames(video_path, output_dir="frames"):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    fps = int(cap.get(cv2.CAP_PROP_FPS) or 30)
    print(f"🎞️ Extracting frames from '{video_path}'...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imwrite(f"{output_dir}/frame_{count:06d}.jpg", frame)
        count += 1

    cap.release()
    print(f"✅ Extracted {count} frames")
    return count, fps


In [5]:
# ===============================
# 🧩 Day 3: Frame Similarity Matrix (Histogram + Flow + SSIM)
# ===============================

def combined_similarity_fast(args):
    pair_idx, frames_resized = args
    i, j = pair_idx
    
    f1, f2 = frames_resized[i], frames_resized[j]

    # 1. Histogram similarity
    h1 = cv2.calcHist([f1], [0,1,2], None, [6,6,6], [0,256]*3)
    h2 = cv2.calcHist([f2], [0,1,2], None, [6,6,6], [0,256]*3)
    cv2.normalize(h1, h1); cv2.normalize(h2, h2)
    hist_score = (cv2.compareHist(h1, h2, cv2.HISTCMP_CORREL) + 1) / 2
    hist_score = np.clip(hist_score, 0, 1)

    # 2. Optical flow
    g1, g2 = cv2.cvtColor(f1, cv2.COLOR_BGR2GRAY), cv2.cvtColor(f2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(g1, g2, None, 0.5, 2, 10, 2, 5, 1.1, 0)
    flow_mag = np.mean(np.sqrt(flow[...,0]**2 + flow[...,1]**2))
    flow_score = np.clip(np.exp(-flow_mag / 10.0), 0, 1)

    # 3. Fast MSE
    f1_tiny = cv2.resize(f1, (80,45))
    f2_tiny = cv2.resize(f2, (80,45))
    mse = np.mean((f1_tiny.astype(np.float32) - f2_tiny.astype(np.float32))**2)
    mse_score = np.clip(np.exp(-mse / 1000.0), 0, 1)

    # Weighted sum
    total = 0.35*hist_score + 0.50*flow_score + 0.15*mse_score
    return i, j, total

def build_similarity_matrix_threaded(frames_resized):
    n = len(frames_resized)
    sim = np.zeros((n, n))
    pairs = [(i, j) for i in range(n) for j in range(i + 1, n)]
    
    print(f"🧮 Computing similarity matrix with {NUM_WORKERS} threads...")
    with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
        args_list = [((i, j), frames_resized) for i, j in pairs]
        results = list(tqdm(executor.map(combined_similarity_fast, args_list),
                            total=len(pairs), ncols=100))
    for i, j, val in results:
        sim[i][j] = sim[j][i] = val
    print("✅ Similarity matrix built")
    return sim



In [6]:
## Day 4 — Optimal Chain Building & Refinements

def build_optimal_chain(similarity, lookahead=3):
    n = len(similarity)
    start_scores = []
    for i in range(n):
        sorted_sims = np.sort(similarity[i])[::-1]
        score = sorted_sims[0] - sorted_sims[1]
        start_scores.append(score)
    start_candidates = np.argsort(start_scores)[-2:]

    best_chain, best_score = None, -999999
    print("🔍 Building optimal chain...")

    for start_frame in start_candidates:
        chain = [start_frame]; visited = {start_frame}
        current, total_score = start_frame, 0

        for step in range(n - 1):
            unvisited = [j for j in range(n) if j not in visited]
            if not unvisited: break

            remaining_frames = len(unvisited)
            end_game_threshold = max(20, int(0.15 * n))

            if remaining_frames <= end_game_threshold:
                best_next = max(unvisited, key=lambda x: similarity[current][x])
            else:
                best_next = None; best_lookahead_score = -999999
                candidates = sorted(unvisited, key=lambda x: similarity[current][x], reverse=True)[:3]
                for candidate in candidates:
                    lookahead_score = similarity[current][candidate]
                    future_unvisited = [j for j in unvisited if j != candidate]
                    if future_unvisited:
                        future_conn = [similarity[candidate][j] for j in future_unvisited]
                        top_future = sorted(future_conn, reverse=True)[:2]
                        lookahead_score += 0.5 * np.mean(top_future)
                    if lookahead_score > best_lookahead_score:
                        best_lookahead_score, best_next = lookahead_score, candidate

            chain.append(best_next)
            visited.add(best_next)
            total_score += similarity[current][best_next]
            current = best_next

        avg_score = total_score / (len(chain) - 1)
        print(f"   Start {start_frame:03d}: Avg similarity = {avg_score:.4f}")
        if avg_score > best_score:
            best_score, best_chain = avg_score, chain

    print(f"✅ Best chain: Start={best_chain[0]:03d}, Avg={best_score:.4f}")
    return best_chain


# Post-processing utilities (fix end, validate flow, remove outliers)
def fix_end_sequence(order, similarity, last_n=20):
    if len(order) <= last_n: return order
    print(f"\n🔧 Re-optimizing last {last_n} frames...")
    fixed_part = order[:-last_n]
    end_frames = set(order[-last_n:])
    current = fixed_part[-1]; new_end = []
    for _ in range(last_n):
        if not end_frames: break
        best = max(end_frames, key=lambda x: similarity[current][x])
        new_end.append(best)
        end_frames.remove(best)
        current = best
    final_order = fixed_part + new_end
    return final_order


def remove_outliers(order, similarity, threshold=0.10):
    n = len(order); clean_order = [order[0]]; removed = 0
    for i in range(1, n):
        prev_idx, curr_idx = order[i-1], order[i]
        sim_score = similarity[prev_idx][curr_idx]
        if sim_score > threshold:
            clean_order.append(curr_idx)
        else:
            removed += 1
    print(f"\n⚠️ Removed {removed} outliers" if removed else "\n✅ No outliers")
    return clean_order


In [7]:
## Day 5 — Rebuilding Video & Main Pipeline

def rebuild_video(order, frame_dir="frames", output_file="reconstructed_video.mp4", fps=30):
    sample = cv2.imread(f"{frame_dir}/frame_000000.jpg")
    h, w, _ = sample.shape
    out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))
    print(f"🎬 Rebuilding video ({len(order)} frames)...")

    def load_frame(idx):
        return cv2.imread(f"{frame_dir}/frame_{idx:06d}.jpg")

    with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
        for frame in tqdm(executor.map(load_frame, order), total=len(order), desc="Writing", ncols=100):
            if frame is not None:
                out.write(frame)

    out.release()
    print(f"✅ Video saved: {output_file}")


def main(video_path):
    start_time = time.time()
    total, fps = extract_frames(video_path)

    print("📥 Loading frames...")
    frames_resized = []
    for i in tqdm(range(total), desc="Loading", ncols=100):
        frame = cv2.imread(f"frames/frame_{i:06d}.jpg")
        frame_resized = cv2.resize(frame, ANALYSIS_RESIZE)
        frames_resized.append(frame_resized)

    similarity = build_similarity_matrix_threaded(frames_resized)
    best_order = build_optimal_chain(similarity)
    best_order = fix_end_sequence(best_order, similarity, last_n=20)
    clean_order = remove_outliers(best_order, similarity, threshold=0.10)
    rebuild_video(clean_order, fps=fps)

    print(f"\n🕒 Total time: {time.time() - start_time:.2f}s")
    print("✅ Done!")


if __name__ == "__main__":
    main("jumbled_video.mp4")


🎞️ Extracting frames from 'jumbled_video.mp4'...
✅ Extracted 300 frames
📥 Loading frames...


Loading: 100%|████████████████████████████████████████████████████| 300/300 [00:05<00:00, 58.79it/s]


🧮 Computing similarity matrix with 32 threads...


100%|████████████████████████████████████████████████████████| 44850/44850 [01:16<00:00, 589.66it/s]


✅ Similarity matrix built
🔍 Building optimal chain...
   Start 293: Avg similarity = 0.9950
   Start 096: Avg similarity = 0.9954
✅ Best chain: Start=096, Avg=0.9954

🔧 Re-optimizing last 20 frames...

✅ No outliers
🎬 Rebuilding video (300 frames)...


Writing: 100%|████████████████████████████████████████████████████| 300/300 [00:08<00:00, 36.85it/s]

✅ Video saved: reconstructed_video.mp4

🕒 Total time: 97.11s
✅ Done!
